# Strength Level overview stats

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tkinter as tk
import datetime
from tkinter import filedialog
import ipywidgets as widgets

pd.options.mode.copy_on_write = True

### Import Strength Level CSV

In [ ]:
root = tk.Tk()
root.withdraw()
root.call("wm", "attributes", ".", "-topmost", True)
file_path = filedialog.askopenfilename(
    title="Select Strength Level CSV", initialdir=os.getcwd()
)
print("Selected file:")
print(file_path)

### Import and preprocess dataframe

In [ ]:
df_SL = pd.read_csv(file_path, sep=",", header=0, index_col=None)
df_SL["Datetime"] = [
    datetime.datetime.strptime(date, "%Y-%m-%d") for date in df_SL["Date Lifted"]
]
df_SL["Year"] = [item.year for item in df_SL["Datetime"]]


### Summary statistics

In [ ]:
n_workouts = len(np.unique(df_SL["Date Lifted"]))
print("Total number of workouts: {}".format(n_workouts))

exercises = np.unique(df_SL["Exercise"])
n_exercises = len(exercises)
print("Number of different exercises performed: {}".format(n_exercises))

date_range = (df_SL["Datetime"].iloc[0] - df_SL["Datetime"].iloc[-1]).days
train_frequency = n_workouts / (date_range / 7)
print("Training frequency: {} workouts per week".format(np.round(train_frequency, 1)))
print(" ")

exercise_count = np.array(
    [
        len(np.unique(df_SL["Date Lifted"][df_SL["Exercise"] == exercise]))
        for exercise in exercises
    ]
)
exercise_sortidx = np.flip(np.argsort(exercise_count))
exercises_sorted = exercises[exercise_sortidx]
exercise_count_sorted = exercise_count[exercise_sortidx]

exercise_sets = np.array(
    [len(df_SL["Date Lifted"][df_SL["Exercise"] == exercise]) for exercise in exercises]
)
exercise_sets_sorted = exercise_sets[exercise_sortidx]

exercise_volume = np.array(
    [
        np.sum(df_SL["Weight (kg)"][df_SL["Exercise"] == exercise])
        for exercise in exercises
    ]
)
exercise_volume_sorted = exercise_volume[exercise_sortidx]

exercise_record = np.array(
    [
        np.max(df_SL["Weight (kg)"][df_SL["Exercise"] == exercise])
        for exercise in exercises
    ]
)
exercise_record_sorted = exercise_record[exercise_sortidx]

exercise_record_bw = (
    exercise_record
    / np.array(
        [
            (
                df_SL["Bodyweight (kg)"][
                    (df_SL["Exercise"] == exercise)
                    & (df_SL["Weight (kg)"] == exercise_record[i])
                ]
            ).min()
            for i, exercise in enumerate(exercises)
        ]
    )
    * 100
)
exercise_record_bw_sorted = exercise_record_bw[exercise_sortidx]

exercise_record_perc = np.array(
    [
        np.max(df_SL["Percentile (%)"][df_SL["Exercise"] == exercise])
        for exercise in exercises
    ]
)
exercise_record_perc_sorted = exercise_record_perc[exercise_sortidx]

n_top = 6
print("Top {} most performed exercises: ".format(n_top))
for i, exercise in enumerate(exercises_sorted[:n_top]):
    print(
        "     {}: \n        {} times, \n        {} kg max weight lifted ({}% bodyweight), \n        {} percentile best set, \n        {} sets, \n        {} kg total volume".format(
            exercise,
            exercise_count_sorted[i],
            exercise_record_sorted[i],
            np.round(exercise_record_bw_sorted[i], 1),
            exercise_record_perc_sorted[i],
            exercise_sets_sorted[i],
            np.round(exercise_volume_sorted[i], 1),
        )
    )

### Progress plot

In [ ]:
fig, axis = plt.subplots(1,2, figsize=(12,5))

for i, exercise in enumerate(exercises_sorted[:n_top]):
    axis[0].plot(df_SL["Datetime"][df_SL["Exercise"] == exercise], df_SL["Weight (kg)"][df_SL["Exercise"] == exercise])
    axis[1].plot(df_SL["Datetime"][df_SL["Exercise"] == exercise], df_SL["Percentile (%)"][df_SL["Exercise"] == exercise])

axis[0].set_xlabel("Date")
axis[0].set_ylabel("Weight (kg)")
axis[0].set_title("Weight")
axis[0].grid()
axis[0].tick_params(axis='x', labelrotation=45)
axis[1].set_xlabel("Date")
axis[1].set_ylabel("Percentile (%)")
axis[1].set_title("Percentile")
axis[1].grid()
axis[1].tick_params(axis='x', labelrotation=45)
fig.legend(exercises_sorted[:n_top], loc="center left", bbox_to_anchor=(0.9, 0.5))

### Summary per year

In [ ]:
years = np.unique(df_SL.loc[:, "Year"])
year_select = widgets.Select(
    options = years,
    value=years[0],
    description="Year to summarise:",
    style={"description_width": "initial"},
)

confirm_button = widgets.Button(description="Confirm", disabled=False)

display(year_select, confirm_button)

def confirm_button_click(a):
    global df_year
    year_selected = year_select.value
    print("Year selected: " + str(year_selected))

    df_year = df_SL.loc[df_SL.loc[:, "Year"] == year_selected, :]

    return(year_selected)

confirm_button.on_click(confirm_button_click)

In [ ]:
n_workouts = len(np.unique(df_year["Date Lifted"]))
print("Total number of workouts: {}".format(n_workouts))

exercises = np.unique(df_year["Exercise"])
n_exercises = len(exercises)
print("Number of different exercises performed: {}".format(n_exercises))

date_range = (df_year["Datetime"].iloc[0] - df_year["Datetime"].iloc[-1]).days
train_frequency = n_workouts / (date_range / 7)
print("Training frequency: {} workouts per week".format(np.round(train_frequency, 1)))
print(" ")

exercise_count = np.array(
    [
        len(np.unique(df_year["Date Lifted"][df_year["Exercise"] == exercise]))
        for exercise in exercises
    ]
)
exercise_sortidx = np.flip(np.argsort(exercise_count))
exercises_sorted = exercises[exercise_sortidx]
exercise_count_sorted = exercise_count[exercise_sortidx]

exercise_sets = np.array(
    [len(df_year["Date Lifted"][df_year["Exercise"] == exercise]) for exercise in exercises]
)
exercise_sets_sorted = exercise_sets[exercise_sortidx]

exercise_volume = np.array(
    [
        np.sum(df_year["Weight (kg)"][df_year["Exercise"] == exercise])
        for exercise in exercises
    ]
)
exercise_volume_sorted = exercise_volume[exercise_sortidx]

exercise_record = np.array(
    [
        np.max(df_year["Weight (kg)"][df_year["Exercise"] == exercise])
        for exercise in exercises
    ]
)
exercise_record_sorted = exercise_record[exercise_sortidx]

exercise_record_bw = (
    exercise_record
    / np.array(
        [
            (
                df_year["Bodyweight (kg)"][
                    (df_year["Exercise"] == exercise)
                    & (df_year["Weight (kg)"] == exercise_record[i])
                ]
            ).min()
            for i, exercise in enumerate(exercises)
        ]
    )
    * 100
)
exercise_record_bw_sorted = exercise_record_bw[exercise_sortidx]

exercise_record_perc = np.array(
    [
        np.max(df_year["Percentile (%)"][df_year["Exercise"] == exercise])
        for exercise in exercises
    ]
)
exercise_record_perc_sorted = exercise_record_perc[exercise_sortidx]

n_top = 6
print("Top {} most performed exercises: ".format(n_top))
for i, exercise in enumerate(exercises_sorted[:n_top]):
    print(
        "     {}: \n        {} times, \n        {} kg max weight lifted ({}% bodyweight), \n        {} percentile best set, \n        {} sets, \n        {} kg total volume".format(
            exercise,
            exercise_count_sorted[i],
            exercise_record_sorted[i],
            np.round(exercise_record_bw_sorted[i], 1),
            exercise_record_perc_sorted[i],
            exercise_sets_sorted[i],
            np.round(exercise_volume_sorted[i], 1),
        )
    )